In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
covid_data = pd.read_csv('covid19_italy_region.csv',parse_dates=['Date'],index_col='Date')
covid_data.head()

,SNo,Country,RegionCode,RegionName,Latitude,Longitude,HospitalizedPatients,IntensiveCarePatients,TotalHospitalizedPatients,HomeConfinement,CurrentPositiveCases,NewPositiveCases,Recovered,Deaths,TotalPositiveCases,TestsPerformed
Date,,,,,,,,,,,,,,,,
2020-02-24 18:00:00,0,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,0,0,0,0,0,NaN
2020-02-24 18:00:00,1,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,0,0,0,0,0,NaN
2020-02-24 18:00:00,2,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,0,0,0,0,0,NaN
2020-02-24 18:00:00,3,ITA,15,Campania,40.839566,14.250850,0,0,0,0,0,0,0,0,0,NaN
2020-02-24 18:00:00,4,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,18,18,0,0,18,NaN


In [3]:
time_series = covid_data['NewPositiveCases']
time_series

Date
2020-02-24 18:00:00       0
2020-02-24 18:00:00       0
2020-02-24 18:00:00       0
2020-02-24 18:00:00       0
2020-02-24 18:00:00      18
                       ... 
2020-12-06 17:00:00    1022
2020-12-06 17:00:00     753
2020-12-06 17:00:00     234
2020-12-06 17:00:00      34
2020-12-06 17:00:00    3444
Name: NewPositiveCases, Length: 6027, dtype: int64

In [4]:
time_series = time_series.groupby('Date').mean()
time_series

Date
2020-02-24 18:00:00      10.523810
2020-02-25 18:00:00       4.428571
2020-02-26 18:00:00       3.714286
2020-02-27 18:00:00      11.904762
2020-02-28 18:00:00      11.333333
                          ...     
2020-12-02 17:00:00     986.142857
2020-12-03 17:00:00    1105.952381
2020-12-04 17:00:00    1147.571429
2020-12-05 17:00:00    1002.476190
2020-12-06 17:00:00     899.380952
Name: NewPositiveCases, Length: 287, dtype: float64

In [15]:
data=time_series.values
data=data.astype('float32')

In [16]:
#normalize data
np.random.seed(7)
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data.reshape(-1,1))

In [19]:
#split the dataset

train_size = int(len(data) * 0.80)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]
print(len(train), len(test))

229 58


In [20]:
# convert an array of values into a dataset matrix

def data_matrix(data,look_back=1):
    X,Y=[],[]
    for i in range(len(data)-look_back-1):
        k=data[i:(i+look_back),0]
        X.append(k)
        Y.append(data[i+look_back,0])
    return np.array(X),np.array(Y)

In [21]:
look_back = 1
X_train, Y_train = data_matrix(train, look_back)
X_test, Y_test = data_matrix(test, look_back)

In [22]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [24]:
# model creation

model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=50, batch_size=1, verbose=2)

Epoch 1/50
227/227 - 3s - loss: 0.0015 - accuracy: 0.0044
Epoch 2/50
227/227 - 1s - loss: 0.0013 - accuracy: 0.0044
Epoch 3/50
227/227 - 1s - loss: 0.0011 - accuracy: 0.0044
Epoch 4/50
227/227 - 1s - loss: 8.5877e-04 - accuracy: 0.0044
Epoch 5/50
227/227 - 0s - loss: 6.4933e-04 - accuracy: 0.0044
Epoch 6/50
227/227 - 0s - loss: 4.4708e-04 - accuracy: 0.0044
Epoch 7/50
227/227 - 0s - loss: 2.8337e-04 - accuracy: 0.0044
Epoch 8/50
227/227 - 0s - loss: 1.7555e-04 - accuracy: 0.0044
Epoch 9/50
227/227 - 0s - loss: 1.0779e-04 - accuracy: 0.0044
Epoch 10/50
227/227 - 0s - loss: 9.0217e-05 - accuracy: 0.0044
Epoch 11/50
227/227 - 0s - loss: 8.1396e-05 - accuracy: 0.0044
Epoch 12/50
227/227 - 0s - loss: 7.9551e-05 - accuracy: 0.0044
Epoch 13/50
227/227 - 0s - loss: 8.2519e-05 - accuracy: 0.0044
Epoch 14/50
227/227 - 0s - loss: 7.7985e-05 - accuracy: 0.0044
Epoch 15/50
227/227 - 0s - loss: 8.8816e-05 - accuracy: 0.0044
Epoch 16/50
227/227 - 0s - loss: 7.8163e-05 - accuracy: 0.0044
Epoch 17/50
2